<a href="https://colab.research.google.com/github/awitz23/Project_CS-GO/blob/main/Nicht_Automatisiert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Installationen

In [1]:
pip install Pillow==9.0.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
%%shell
add-apt-repository -y ppa:longsleep/golang-backports
apt -y update
apt -y install golang-go

pip install awpy

#wget https://storage.googleapis.com/csgo-tests/default.dem

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/longsleep/golang-ba

In [3]:
import pandas as pd
import numpy as np

#from awpy.parser import DemoParser
from awpy import *
#from awpy.data import NAV_CSV --> evtl. nicht notwendig, mal ausprobieren

from google.colab import drive
from google.colab import files
drive.mount('/content/drive')


from awpy.data import NAV_CSV

!pip install nashpy
import nashpy as nash
import numpy as np

pd.set_option('max_rows', 200)
pd.set_option('max_colwidth', 4000)
pd.describe_option('max_colwidth')
df_strategy=pd.DataFrame()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
display.max_colwidth : int or None
    The maximum width in characters of a column in the repr of
    a pandas data structure. When the column overflows, a "..."
    placeholder is embedded in the output. A 'None' value means unlimited.
    [default: 50] [currently: 4000]


#Parsing

In [4]:
p = DemoParser(demofile="/content/drive/MyDrive/Demofiles/vitality-vs-liquid-m2-dust2.dem", parse_rate=128)
print(p)
#d = p.parse()
#d.keys()
df = p.parse(return_type="df")
#df.keys()

#Versuch mit Parsing über alle Runden T-Seite

In [7]:
p = DemoParser(demofile="/content/drive/MyDrive/Demofiles/natus-vincere-vs-ence-m2-dust2.dem", parse_rate=128)

#d = p.parse()
#d.keys()
df = p.parse(return_type="df")
#df.keys()

In [8]:
i = 1
Gameround = []
Strategies = []
while i <= df["playerFrames"]["roundNum"].max():
  player= df["playerFrames"]
  track3 = player
  track4 = track3[(track3['seconds'] > 10) & (track3['seconds'] <= 30)]
  track4 = track4.loc[track4["roundNum"]==i] 
  track4 = track4.loc[track4["side"]=="T"]
  
  t = NAV_CSV[NAV_CSV["mapName"] == "de_dust2"]


  area = []
  #für jede Reihe durchgehen und Position festlegen und in Liste einspeichern --> jede Sekunde für jeden Spieler
  for index, row in track4.iterrows():
    f = t[(t['northWestX'] > row['x']-40) & (t['northWestX'] < row['x']+40)]
    f = f[(f['northWestY'] > row['y']-40) & (f['northWestY'] < row['y']+40)]
  
    if f.empty:
      loc = "MISTAKE"
      area.append(loc)
    
    else:
      loc = f["areaName"].value_counts().idxmax()
      area.append(loc)

  #füge Liste mit Positionen zu DF hinzu
  track4["area"] = area 
  
  track4.loc[track4['area'] == "CTSpawn",'subgroup'] = "Mid"
  track4.loc[track4['area'] == "MidDoors",'subgroup'] = "Mid"
  track4.loc[track4['area'] == "TopofMid",'subgroup'] = "Mid"
  track4.loc[track4['area'] == "Middle",'subgroup'] = "Mid"
  track4.loc[track4['area'] == "Short",'subgroup'] = "Mid"

  track4.loc[track4['area'] == "ShortStairs",'subgroup'] = "BombA"
  track4.loc[track4['area'] == "UnderA",'subgroup'] = "BombA"
  track4.loc[track4['area'] == "ExtendedA",'subgroup'] = "BombA"
  track4.loc[track4['area'] == "Catwalk",'subgroup'] = "BombA"

  track4.loc[track4['area'] == "BombsiteB",'subgroup'] = "BombB"
  track4.loc[track4['area'] == "BDoors",'subgroup'] = "BombB"
  track4.loc[track4['area'] == "Hole",'subgroup'] = "BombB"

  track4.loc[track4['area'] == "TSpawn",'subgroup'] = "TerroristSpawn"
  track4.loc[track4['area'] == "TRamp",'subgroup'] = "TerroristSpawn"

  track4.loc[track4['area'] == "OutsideTunnel",'subgroup'] = "BombB"
  track4.loc[track4['area'] == "UpperTunnel",'subgroup'] = "BombB"
  track4.loc[track4['area'] == "TunnelStairs",'subgroup'] = "BombB"
  track4.loc[track4['area'] == "LowerTunnel",'subgroup'] = "BombB"

  track4.loc[track4['area'] == "LongA",'subgroup'] = "BombA"
  track4.loc[track4['area'] == "LongDoors",'subgroup'] = "BombA"
  track4.loc[track4['area'] == "OutsideLong",'subgroup'] = "BombA"
  track4.loc[track4['area'] == "Pit",'subgroup'] = "BombA"
  track4.loc[track4['area'] == "Ramp",'subgroup'] = "BombA"
  track4.loc[track4['area'] == "Side",'subgroup'] = "BombA"

  track4.loc[track4['area'] == "BombsiteA",'subgroup'] = "BombA"
  track4.loc[track4['area'] == "ARamp",'subgroup'] = "BombA"

  stra = []

  df2 = track4.groupby(['subgroup', 'name'])['subgroup'].count().unstack(fill_value=0)

  for column in df2:
    y = df2[{column}].idxmax()
    y = y.values.tolist()
    stra.append(y)
  
  j = 0
  

  BombA = 0
  BombB = 0
  Mid = 0
  TerroristSpawn = 0

  for element in stra:
    if (element[j] == "BombA"):
      BombA = BombA+1
    elif (element[j] == "BombB"):
      BombB = BombB+1
    elif (element[j] == "Mid"):
      Mid = Mid+1
    elif (element[j] == "Terroristspawn"):
      TerroristSpawn = TerroristSpawn + 1 
  if BombA >= 4:
      strategy = "RushA"
      Strategies.append(strategy)
      strategy=0
      Gameround.append(i)
  if BombB >=4:
      strategy = "RushB"
      Strategies.append(strategy)
      strategy=0
      Gameround.append(i)
  if (BombA<4)&(BombB<4):
      strategy = "Split"
      Strategies.append(strategy)
      strategy=0
      Gameround.append(i)
  print(i)
  #print(stra)
  print(Strategies)

  i=i+1


1
['RushB']
2
['RushB', 'Split']
3
['RushB', 'Split', 'RushA']
4
['RushB', 'Split', 'RushA', 'Split']
5
['RushB', 'Split', 'RushA', 'Split', 'Split']
6
['RushB', 'Split', 'RushA', 'Split', 'Split', 'Split']
7
['RushB', 'Split', 'RushA', 'Split', 'Split', 'Split', 'RushB']
8
['RushB', 'Split', 'RushA', 'Split', 'Split', 'Split', 'RushB', 'Split']
9
['RushB', 'Split', 'RushA', 'Split', 'Split', 'Split', 'RushB', 'Split', 'RushA']
10
['RushB', 'Split', 'RushA', 'Split', 'Split', 'Split', 'RushB', 'Split', 'RushA', 'RushB']
11
['RushB', 'Split', 'RushA', 'Split', 'Split', 'Split', 'RushB', 'Split', 'RushA', 'RushB', 'RushB']
12
['RushB', 'Split', 'RushA', 'Split', 'Split', 'Split', 'RushB', 'Split', 'RushA', 'RushB', 'RushB', 'Split']
13
['RushB', 'Split', 'RushA', 'Split', 'Split', 'Split', 'RushB', 'Split', 'RushA', 'RushB', 'RushB', 'Split', 'Split']
14
['RushB', 'Split', 'RushA', 'Split', 'Split', 'Split', 'RushB', 'Split', 'RushA', 'RushB', 'RushB', 'Split', 'Split', 'Split']
15
['Rus

In [9]:
i = 1
Gameround = []
CTStrategies = []
while i <= df["playerFrames"]["roundNum"].max():
  player= df["playerFrames"]
  track3 = player
  track4 = track3[(track3['seconds'] > 10) & (track3['seconds'] <= 30)]
  track4 = track4.loc[track4["roundNum"]==i] 
  track4 = track4.loc[track4["side"]=="CT"]
  
  t = NAV_CSV[NAV_CSV["mapName"] == "de_dust2"]


  area = []
  #für jede Reihe durchgehen und Position festlegen und in Liste einspeichern --> jede Sekunde für jeden Spieler
  for index, row in track4.iterrows():
    f = t[(t['northWestX'] > row['x']-80) & (t['northWestX'] < row['x']+80)]
    f = f[(f['northWestY'] > row['y']-80) & (f['northWestY'] < row['y']+80)]
  
    if f.empty:
      loc = "MISTAKE"
      area.append(loc)
    
    else:
      loc = f["areaName"].value_counts().idxmax()
      area.append(loc)

  #füge Liste mit Positionen zu DF hinzu
  track4["area"] = area 
  
  track4.loc[track4['area'] == "CTSpawn",'subgroup'] = "Mid"
  track4.loc[track4['area'] == "MidDoors",'subgroup'] = "Mid"
  track4.loc[track4['area'] == "TopofMid",'subgroup'] = "Mid"
  track4.loc[track4['area'] == "Middle",'subgroup'] = "Mid"
  track4.loc[track4['area'] == "Short",'subgroup'] = "Mid"

  track4.loc[track4['area'] == "ShortStairs",'subgroup'] = "BombA"
  track4.loc[track4['area'] == "UnderA",'subgroup'] = "BombA"
  track4.loc[track4['area'] == "ExtendedA",'subgroup'] = "BombA"
  track4.loc[track4['area'] == "Catwalk",'subgroup'] = "BombA"

  track4.loc[track4['area'] == "BombsiteB",'subgroup'] = "BombB"
  track4.loc[track4['area'] == "BDoors",'subgroup'] = "BombB"
  track4.loc[track4['area'] == "Hole",'subgroup'] = "BombB"

  track4.loc[track4['area'] == "TSpawn",'subgroup'] = "TerroristSpawn"
  track4.loc[track4['area'] == "TRamp",'subgroup'] = "TerroristSpawn"

  track4.loc[track4['area'] == "OutsideTunnel",'subgroup'] = "BombB"
  track4.loc[track4['area'] == "UpperTunnel",'subgroup'] = "BombB"
  track4.loc[track4['area'] == "TunnelStairs",'subgroup'] = "BombB"
  track4.loc[track4['area'] == "LowerTunnel",'subgroup'] = "BombB"

  track4.loc[track4['area'] == "LongA",'subgroup'] = "BombA"
  track4.loc[track4['area'] == "LongDoors",'subgroup'] = "BombA"
  track4.loc[track4['area'] == "OutsideLong",'subgroup'] = "BombA"
  track4.loc[track4['area'] == "Pit",'subgroup'] = "BombA"
  track4.loc[track4['area'] == "Ramp",'subgroup'] = "BombA"
  track4.loc[track4['area'] == "Side",'subgroup'] = "BombA"

  track4.loc[track4['area'] == "BombsiteA",'subgroup'] = "BombA"
  track4.loc[track4['area'] == "ARamp",'subgroup'] = "BombA"

  stra = []

  df2 = track4.groupby(['subgroup', 'name'])['subgroup'].count().unstack(fill_value=0)

  for column in df2:
    y = df2[{column}].idxmax()
    y = y.values.tolist()
    stra.append(y)
  
  j = 0
  

  BombA = 0
  BombB = 0
  Mid = 0
  TerroristSpawn = 0

  for element in stra:
    if (element[j] == "BombA"):
      BombA = BombA+1
    elif (element[j] == "BombB"):
      BombB = BombB+1
    elif (element[j] == "Mid"):
      Mid = Mid+1
    elif (element[j] == "Terroristspawn"):
      TerroristSpawn = TerroristSpawn + 1 
  if BombA >= 3:
      strategy = "FullA"
      CTStrategies.append(strategy)
      strategy=0
      Gameround.append(i)
  if BombB >=3:
      strategy = "FullB"
      CTStrategies.append(strategy)
      strategy=0
      Gameround.append(i)
  if (BombA<3)&(BombB<3):
      strategy = "Split"
      CTStrategies.append(strategy)
      strategy=0
      Gameround.append(i)
  print(i)
  #print(stra)
  print(CTStrategies)

  i=i+1
  


1
['Split']
2
['Split', 'Split']
3
['Split', 'Split', 'FullA']
4
['Split', 'Split', 'FullA', 'FullA']
5
['Split', 'Split', 'FullA', 'FullA', 'FullB']
6
['Split', 'Split', 'FullA', 'FullA', 'FullB', 'FullB']
7
['Split', 'Split', 'FullA', 'FullA', 'FullB', 'FullB', 'FullB']
8
['Split', 'Split', 'FullA', 'FullA', 'FullB', 'FullB', 'FullB', 'FullA']
9
['Split', 'Split', 'FullA', 'FullA', 'FullB', 'FullB', 'FullB', 'FullA', 'FullA']
10
['Split', 'Split', 'FullA', 'FullA', 'FullB', 'FullB', 'FullB', 'FullA', 'FullA', 'FullA']
11
['Split', 'Split', 'FullA', 'FullA', 'FullB', 'FullB', 'FullB', 'FullA', 'FullA', 'FullA', 'FullA']
12
['Split', 'Split', 'FullA', 'FullA', 'FullB', 'FullB', 'FullB', 'FullA', 'FullA', 'FullA', 'FullA', 'Split']
13
['Split', 'Split', 'FullA', 'FullA', 'FullB', 'FullB', 'FullB', 'FullA', 'FullA', 'FullA', 'FullA', 'Split', 'Split']
14
['Split', 'Split', 'FullA', 'FullA', 'FullB', 'FullB', 'FullB', 'FullA', 'FullA', 'FullA', 'FullA', 'Split', 'Split', 'FullA']
15
['Spl

In [10]:
Winner = df["rounds"]["winningSide"].tolist()
print(Winner)

['T', 'T', 'CT', 'T', 'T', 'T', 'T', 'T', 'CT', 'T', 'T', 'T', 'T', 'T', 'T', 'CT', 'T', 'CT', 'CT']


In [11]:

df_strategy_temp = pd.DataFrame({'Round':Gameround, "T-strategy":Strategies, "CT-strategy":CTStrategies, "Winner":Winner})
df_strategy = df_strategy.append(df_strategy_temp)

print(df_strategy)


    Round T-strategy CT-strategy Winner
0       1      RushB       Split      T
1       2      Split       Split      T
2       3      RushA       FullA     CT
3       4      Split       FullA      T
4       5      Split       FullB      T
5       6      Split       FullB      T
6       7      RushB       FullB      T
7       8      Split       FullA      T
8       9      RushA       FullA     CT
9      10      RushB       FullA      T
10     11      RushB       FullA      T
11     12      Split       Split      T
12     13      Split       Split      T
13     14      Split       FullA      T
14     15      Split       Split      T
15     16      RushB       FullB     CT
16     17      Split       Split      T
17     18      Split       Split     CT
18     19      RushA       FullA     CT


#Gewinnerseite

#CT Seite

#Strategie DF

In [ ]:
#df_strategy
#df_sicherungskopie=df_strategy
#dfkopie4=df_strategy
#dfkopie4.to_csv("save4.csv")

In [ ]:
#df_sicherungskopie=df_strategy
#df_sicherungskopie

#Weiterversuch

pi_SS = 0
pi_SA = 0
pi_SB = 0
pi_AS = 0
pi_AA = 0
pi_AB = 0
pi_BS = 0
pi_BA = 0
pi_BB = 0

kann man evtl mal gebrauchen wieder

In [12]:
#Aufteilung des großen DF in 9 kleine --> möglichkeit, über shape[0] Zahlen zu bekommen

pi_SS = df_strategy[(df_strategy['T-strategy'] == "Split") & (df_strategy['CT-strategy'] == "Split") & (df_strategy['Winner'] == "T")]
pi_SA = df_strategy[(df_strategy['T-strategy'] == "Split") & (df_strategy['CT-strategy'] == "FullA") & (df_strategy['Winner'] == "T")]
pi_SB = df_strategy[(df_strategy['T-strategy'] == "Split") & (df_strategy['CT-strategy'] == "FullB") & (df_strategy['Winner'] == "T")]
pi_AS = df_strategy[(df_strategy['T-strategy'] == "RushA") & (df_strategy['CT-strategy'] == "Split") & (df_strategy['Winner'] == "T")]
pi_AA = df_strategy[(df_strategy['T-strategy'] == "RushA") & (df_strategy['CT-strategy'] == "FullA") & (df_strategy['Winner'] == "T")]
pi_AB = df_strategy[(df_strategy['T-strategy'] == "RushA") & (df_strategy['CT-strategy'] == "FullB") & (df_strategy['Winner'] == "T")]
pi_BS = df_strategy[(df_strategy['T-strategy'] == "RushB") & (df_strategy['CT-strategy'] == "Split") & (df_strategy['Winner'] == "T")]
pi_BA = df_strategy[(df_strategy['T-strategy'] == "RushB") & (df_strategy['CT-strategy'] == "FullA") & (df_strategy['Winner'] == "T")]
pi_BB = df_strategy[(df_strategy['T-strategy'] == "RushB") & (df_strategy['CT-strategy'] == "FullB") & (df_strategy['Winner'] == "T")]

In [13]:
#gesamtzahl an siegen der T-Seite
wins_T =(
pi_SS.shape[0]+
pi_SA.shape[0]+
pi_SB.shape[0]+
pi_AS.shape[0]+
pi_AA.shape[0]+
pi_AB.shape[0]+
pi_BS.shape[0]+
pi_BA.shape[0]+
pi_BB.shape[0])
print(wins_T)

14


In [14]:

CT_pi_SS = df_strategy[(df_strategy['T-strategy'] == "Split") & (df_strategy['CT-strategy'] == "Split") & (df_strategy['Winner'] == "CT")]
CT_pi_SA = df_strategy[(df_strategy['T-strategy'] == "Split") & (df_strategy['CT-strategy'] == "FullA") & (df_strategy['Winner'] == "CT")]
CT_pi_SB = df_strategy[(df_strategy['T-strategy'] == "Split") & (df_strategy['CT-strategy'] == "FullB") & (df_strategy['Winner'] == "CT")]
CT_pi_AS = df_strategy[(df_strategy['T-strategy'] == "RushA") & (df_strategy['CT-strategy'] == "Split") & (df_strategy['Winner'] == "CT")]
CT_pi_AA = df_strategy[(df_strategy['T-strategy'] == "RushA") & (df_strategy['CT-strategy'] == "FullA") & (df_strategy['Winner'] == "CT")]
CT_pi_AB = df_strategy[(df_strategy['T-strategy'] == "RushA") & (df_strategy['CT-strategy'] == "FullB") & (df_strategy['Winner'] == "CT")]
CT_pi_BS = df_strategy[(df_strategy['T-strategy'] == "RushB") & (df_strategy['CT-strategy'] == "Split") & (df_strategy['Winner'] == "CT")]
CT_pi_BA = df_strategy[(df_strategy['T-strategy'] == "RushB") & (df_strategy['CT-strategy'] == "FullA") & (df_strategy['Winner'] == "CT")]
CT_pi_BB = df_strategy[(df_strategy['T-strategy'] == "RushB") & (df_strategy['CT-strategy'] == "FullB") & (df_strategy['Winner'] == "CT")]

In [15]:
#Gesamtzahl der Ereignisse 
Full_pi_SS = CT_pi_SS.shape[0] + pi_SS.shape[0]
Full_pi_SA = CT_pi_SA.shape[0] + pi_SA.shape[0]
Full_pi_SB = CT_pi_SB.shape[0] + pi_SB.shape[0]
Full_pi_AS = CT_pi_AS.shape[0] + pi_AS.shape[0]
Full_pi_AA = CT_pi_AA.shape[0] + pi_AA.shape[0]
Full_pi_AB = CT_pi_AB.shape[0] + pi_AB.shape[0]
Full_pi_BS = CT_pi_BS.shape[0] + pi_BS.shape[0]
Full_pi_BA = CT_pi_BA.shape[0] + pi_BA.shape[0]
Full_pi_BB = CT_pi_BB.shape[0] + pi_BB.shape[0]

print(Full_pi_AS)

0


In [16]:
wins_CT =(
CT_pi_SS.shape[0]+
CT_pi_SA.shape[0]+
CT_pi_SB.shape[0]+
CT_pi_AS.shape[0]+
CT_pi_AA.shape[0]+
CT_pi_AB.shape[0]+
CT_pi_BS.shape[0]+
CT_pi_BA.shape[0]+
CT_pi_BB.shape[0])
print(wins_CT)

5


In [17]:
import plotly.graph_objects as go

headerColor = 'grey'
rowEvenColor = 'lightgrey'
rowOddColor = 'white'

fig = go.Figure(data=[go.Table(
  header=dict(
    values=['<b> </b>','<b>Split</b>','<b>FullA</b>','<b>FullB</b>','<b>Total</b>'],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=[
      ['Split', 'RushA', 'RushB', '<b>TOTAL</b>'],
      [pi_SS.shape[0], pi_AS.shape[0], pi_BS.shape[0], pi_SS.shape[0]+ pi_AS.shape[0]+ pi_BS.shape[0]],
      [pi_SA.shape[0], pi_AA.shape[0], pi_BA.shape[0], pi_SA.shape[0]+ pi_AA.shape[0]+ pi_BA.shape[0]],
      [pi_SB.shape[0], pi_AB.shape[0], pi_BB.shape[0], pi_SB.shape[0]+ pi_AB.shape[0]+ pi_BB.shape[0]],
      [pi_SS.shape[0]+ pi_SA.shape[0]+ pi_SB.shape[0], pi_AS.shape[0]+ pi_AA.shape[0]+ pi_AB.shape[0],
       pi_BS.shape[0]+ pi_BA.shape[0]+ pi_BB.shape[0], pi_SS.shape[0]+ pi_AS.shape[0]+ pi_BS.shape[0] +
       pi_SA.shape[0]+ pi_AA.shape[0]+ pi_BA.shape[0]+
       pi_SB.shape[0]+ pi_AB.shape[0]+ pi_BB.shape[0]]],
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*5],
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 11)
    ))
])

print("Siege der T Seite")
fig.show()

Siege der T Seite


In [18]:
headerColor = 'grey'
rowEvenColor = 'lightgrey'
rowOddColor = 'white'

fig = go.Figure(data=[go.Table(
  header=dict(
    values=['<b> </b>','<b>Split</b>','<b>FullA</b>','<b>FullB</b>','<b>Total</b>'],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=[
      ['Split', 'RushA', 'RushB', '<b>TOTAL</b>'],
      [Full_pi_SS , Full_pi_AS , Full_pi_BS , Full_pi_SS + Full_pi_AS + Full_pi_BS ],
      [Full_pi_SA , Full_pi_AA , Full_pi_BA , Full_pi_SA + Full_pi_AA + Full_pi_BA ],
      [Full_pi_SB , Full_pi_AB , Full_pi_BB , Full_pi_SB + Full_pi_AB + Full_pi_BB ],
      [Full_pi_SS + Full_pi_SA + Full_pi_SB , Full_pi_AS + Full_pi_AA + Full_pi_AB ,
       Full_pi_BS + Full_pi_BA + Full_pi_BB , Full_pi_SS + Full_pi_AS + Full_pi_BS  +
       Full_pi_SA + Full_pi_AA + Full_pi_BA +
       Full_pi_SB + Full_pi_AB + Full_pi_BB ]],
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*5],
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 11)
    ))
])

print("Gesamtzahl")
fig.show()

Gesamtzahl


In [19]:
headerColor = 'grey'
rowEvenColor = 'lightgrey'
rowOddColor = 'white'

fig = go.Figure(data=[go.Table(
  header=dict(
    values=['<b> </b>','<b>Split</b>','<b>FullA</b>','<b>FullB</b>','<b>Total</b>'],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=[
      ['Split', 'RushA', 'RushB', '<b>TOTAL</b>'],
      [pi_SS.shape[0]/Full_pi_SS, pi_AS.shape[0]/Full_pi_AS, pi_BS.shape[0]/Full_pi_BS, pi_SS.shape[0]+ pi_AS.shape[0]+ pi_BS.shape[0]],
      [pi_SA.shape[0]/Full_pi_SA, pi_AA.shape[0]/Full_pi_AA, pi_BA.shape[0]/Full_pi_BA, pi_SA.shape[0]+ pi_AA.shape[0]+ pi_BA.shape[0]],
      [pi_SB.shape[0]/Full_pi_SB, pi_AB.shape[0]/Full_pi_AB, pi_BB.shape[0]/Full_pi_BB, pi_SB.shape[0]+ pi_AB.shape[0]+ pi_BB.shape[0]],
      [pi_SS.shape[0]+ pi_SA.shape[0]+ pi_SB.shape[0], pi_AS.shape[0]+ pi_AA.shape[0]+ pi_AB.shape[0],
       pi_BS.shape[0]+ pi_BA.shape[0]+ pi_BB.shape[0], pi_SS.shape[0]+ pi_AS.shape[0]+ pi_BS.shape[0] +
       pi_SA.shape[0]+ pi_AA.shape[0]+ pi_BA.shape[0]+
       pi_SB.shape[0]+ pi_AB.shape[0]+ pi_BB.shape[0]]],
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*5],
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 11)
    ))
])

print("SiegWKEIT der T Seite")
fig.show()

ZeroDivisionError: ignored

In [ ]:
headerColor = 'grey'
rowEvenColor = 'lightgrey'
rowOddColor = 'white'

fig = go.Figure(data=[go.Table(
  header=dict(
    values=['<b> </b>','<b>Split</b>','<b>FullA</b>','<b>FullB</b>','<b>Total</b>'],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=[
      ['Split', 'RushA', 'RushB', '<b>TOTAL</b>'],
      [CT_pi_SS.shape[0], CT_pi_AS.shape[0], CT_pi_BS.shape[0], CT_pi_SS.shape[0]+ CT_pi_AS.shape[0]+ CT_pi_BS.shape[0]],
      [CT_pi_SA.shape[0], CT_pi_AA.shape[0], CT_pi_BA.shape[0], CT_pi_SA.shape[0]+ CT_pi_AA.shape[0]+ CT_pi_BA.shape[0]],
      [CT_pi_SB.shape[0], CT_pi_AB.shape[0], CT_pi_BB.shape[0], CT_pi_SB.shape[0]+ CT_pi_AB.shape[0]+ CT_pi_BB.shape[0]],
      [CT_pi_SS.shape[0]+ CT_pi_SA.shape[0]+ CT_pi_SB.shape[0], CT_pi_AS.shape[0]+ CT_pi_AA.shape[0]+ CT_pi_AB.shape[0],
       CT_pi_BS.shape[0]+ CT_pi_BA.shape[0]+ CT_pi_BB.shape[0], CT_pi_SS.shape[0]+ CT_pi_AS.shape[0]+ CT_pi_BS.shape[0] +
       CT_pi_SA.shape[0]+ CT_pi_AA.shape[0]+ CT_pi_BA.shape[0]+
       CT_pi_SB.shape[0]+ CT_pi_AB.shape[0]+ CT_pi_BB.shape[0]]],
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*5],
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 11)
    ))
])

print("Siege der CT Seite")
fig.show()

In [ ]:
headerColor = 'grey'
rowEvenColor = 'lightgrey'
rowOddColor = 'white'

fig = go.Figure(data=[go.Table(
  header=dict(
    values=['<b> </b>','<b>Split</b>','<b>FullA</b>','<b>FullB</b>','<b>Total</b>'],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=[
      ['Split', 'RushA', 'RushB', '<b>TOTAL</b>'],
      [CT_pi_SS.shape[0]/Full_pi_SS, CT_pi_AS.shape[0]/Full_pi_AS, CT_pi_BS.shape[0]/Full_pi_BS, (CT_pi_SS.shape[0]+ CT_pi_AS.shape[0]+ CT_pi_BS.shape[0])/df_strategy.shape[0]],
      [CT_pi_SA.shape[0]/Full_pi_SA, CT_pi_AA.shape[0]/Full_pi_AA, CT_pi_BA.shape[0]/Full_pi_BA, (CT_pi_SA.shape[0]+ CT_pi_AA.shape[0]+ CT_pi_BA.shape[0])/df_strategy.shape[0]],
      [CT_pi_SB.shape[0]/Full_pi_SB, CT_pi_AB.shape[0]/Full_pi_AB, CT_pi_BB.shape[0]/Full_pi_BB, (CT_pi_SB.shape[0]+ CT_pi_AB.shape[0]+ CT_pi_BB.shape[0])/df_strategy.shape[0]],
      [(CT_pi_SS.shape[0]+ CT_pi_SA.shape[0]+ CT_pi_SB.shape[0])/df_strategy.shape[0], (CT_pi_AS.shape[0]+ CT_pi_AA.shape[0]+ CT_pi_AB.shape[0])/df_strategy.shape[0],
       (CT_pi_BS.shape[0]+ CT_pi_BA.shape[0]+ CT_pi_BB.shape[0])/df_strategy.shape[0], (CT_pi_SS.shape[0]+ CT_pi_AS.shape[0]+ CT_pi_BS.shape[0] +
       CT_pi_SA.shape[0]+ CT_pi_AA.shape[0]+ CT_pi_BA.shape[0]+
       CT_pi_SB.shape[0]+ CT_pi_AB.shape[0]+ CT_pi_BB.shape[0])/df_strategy.shape[0]]],
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*5],
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 11)
    ))
])

print("SiegWKEIT der CT Seite")
fig.show()

#Nash Test

In [ ]:

A = np.array([[pi_SS.shape[0]/Full_pi_SS, pi_SA.shape[0]/Full_pi_SA, pi_SB.shape[0]/Full_pi_SB], 
              [pi_AS.shape[0]/Full_pi_AS, pi_AA.shape[0]/Full_pi_AA, pi_AB.shape[0]/Full_pi_AB],
              [pi_BS.shape[0]/Full_pi_BS, pi_BA.shape[0]/Full_pi_BA, pi_BB.shape[0]/Full_pi_BB]])

B = np.array([[CT_pi_SS.shape[0]/Full_pi_SS, CT_pi_SA.shape[0]/Full_pi_SA, CT_pi_SB.shape[0]/Full_pi_SB], 
              [CT_pi_AS.shape[0]/Full_pi_AS, CT_pi_AA.shape[0]/Full_pi_AA, CT_pi_AB.shape[0]/Full_pi_AB],
              [CT_pi_BS.shape[0]/Full_pi_BS, CT_pi_BA.shape[0]/Full_pi_BA, CT_pi_BB.shape[0]/Full_pi_BB]])

#B = np.array([[CT_pi_SS.shape[0]/Full_pi_SS, CT_pi_AS.shape[0]/Full_pi_AS, CT_pi_BS.shape[0]/Full_pi_BS], 
#              [CT_pi_SA.shape[0]/Full_pi_SA, CT_pi_AA.shape[0]/Full_pi_AA, CT_pi_BA.shape[0]/Full_pi_BA],
#              [CT_pi_SB.shape[0]/Full_pi_SB, CT_pi_AB.shape[0]/Full_pi_AB, CT_pi_BB.shape[0]/Full_pi_BB]])
CSGO = nash.Game(A, B)
CSGO

In [ ]:
equilibria = CSGO.support_enumeration()
for eq in equilibria:
    print(eq)

#AB HIER: Versuch 
1. Spieler lokalisieren (X & Y Koord. Suchen)
2. Genauen Bereich lokalisieren (Durch NAV File)

In [ ]:
#1. Spieler lokalisieren und erste 20 sek. tracken

player= df["playerFrames"]
track3 = player

track4 = track3[(track3['seconds'] > 0) & (track3['seconds'] <= 20)]
track4 = track4.loc[track4["roundNum"]==17] 
track4 = track4.loc[track4["side"]=="T"]

In [ ]:
#2. Suche Position d. Spielers in NAV Datei

#ziehe Daten der Map
#t = NAV_CSV[NAV_CSV["mapName"] == "de_dust2"]

#loaklisiere Spieler
#f = t[(t['northWestX'] > track5.iloc[0]["x"]-80) & (t['northWestX'] < track5.iloc[0]["x"]+80)]
#f = f[(f['northWestY'] > track5.iloc[0]["y"]-80) & (f['northWestY'] < track5.iloc[0]["y"]+80)]

#nehme nur eine Position, die die am häufigsten auftritt, da durch Z-Koord. diese häufiger auftreten. Durch höchste Anzahl sollte die richtige koord. gewählt werden
#loc = f["areaName"].value_counts().idxmax()



In [ ]:
#track5 = track5.reset_index() 

#Finde Positionen für alle Spieler in den ersten 20 sekunden zu jeder sekunde

#initialisiere leere Liste um später Positionen einzuspeichern
area = []
#für jede Reihe durchgehen und Position festlegen und in Liste einspeichern --> jede Sekunde für jeden Spieler
for index, row in track4.iterrows():
  f = t[(t['northWestX'] > row['x']-80) & (t['northWestX'] < row['x']+80)]
  f = f[(f['northWestY'] > row['y']-80) & (f['northWestY'] < row['y']+80)]
  
  if f.empty:
    loc = "MISTAKE"
    area.append(loc)
    
  else:
    loc = f["areaName"].value_counts().idxmax()
    area.append(loc)

#füge Liste mit Positionen zu DF hinzu
track4["area"] = area 

    

In [ ]:
#Group into Subgroups
#merke: Short und Long in A umwandeln ! Tunnel in B
track4.loc[track4['area'] == "CTSpawn",'subgroup'] = "Mid"
track4.loc[track4['area'] == "MidDoors",'subgroup'] = "BombB"
track4.loc[track4['area'] == "TopofMid",'subgroup'] = "Mid"
track4.loc[track4['area'] == "Middle",'subgroup'] = "Mid"
track4.loc[track4['area'] == "Short",'subgroup'] = "Mid"

track4.loc[track4['area'] == "ShortStairs",'subgroup'] = "BombA"
track4.loc[track4['area'] == "UnderA",'subgroup'] = "BombA"
track4.loc[track4['area'] == "ExtendedA",'subgroup'] = "BombA"
track4.loc[track4['area'] == "Catwalk",'subgroup'] = "BombA"

track4.loc[track4['area'] == "BombsiteB",'subgroup'] = "BombB"
track4.loc[track4['area'] == "BDoors",'subgroup'] = "BombB"
track4.loc[track4['area'] == "Hole",'subgroup'] = "BombB"

track4.loc[track4['area'] == "TSpawn",'subgroup'] = "TerroristSpawn"
track4.loc[track4['area'] == "TRamp",'subgroup'] = "TerroristSpawn"

track4.loc[track4['area'] == "OutsideTunnel",'subgroup'] = "BombB"
track4.loc[track4['area'] == "UpperTunnel",'subgroup'] = "BombB"
track4.loc[track4['area'] == "TunnelStairs",'subgroup'] = "BombB"
track4.loc[track4['area'] == "LowerTunnel",'subgroup'] = "BombB"

track4.loc[track4['area'] == "LongA",'subgroup'] = "BombA"
track4.loc[track4['area'] == "LongDoors",'subgroup'] = "BombA"
track4.loc[track4['area'] == "OutsideLong",'subgroup'] = "BombA"
track4.loc[track4['area'] == "Pit",'subgroup'] = "BombA"
track4.loc[track4['area'] == "Side",'subgroup'] = "BombA"

track4.loc[track4['area'] == "BombsiteA",'subgroup'] = "BombA"
track4.loc[track4['area'] == "ARamp",'subgroup'] = "BombA"
track4.loc[track4['area'] == "Ramp",'subgroup'] = "BombA"

#df.loc[df['days'] <=20,'subgroup'] = 3
#df.loc[df['days'] <=10,'subgroup'] = 2
#df.loc[df['days'] ==0,'subgroup'] = 1
#df = df.reset_index()
#df['dynamic_subgroup'] = df.groupby(['group'])['subgroup'].rank(method='dense')
#track4

In [ ]:
#zeigt count der orte der ersten 20 sekunden der spieler an

stra = []

df2 = track4.groupby(['subgroup', 'name'])['subgroup'].count().unstack(fill_value=0)
print(df2)

In [ ]:
#zeige, welche Position von welchem Spieler am häufigsten innerhalb der Zeit besetzt war

for column in df2:
  y = df2[{column}].idxmax()
  y = y.values.tolist()
  stra.append(y)

In [ ]:
##zähle wie oft welches element in der Liste stra vorhanden war und leite davon die Strategie ab
print(stra)
j = 0
Strategies = []

BombA = 0
BombB = 0
Mid = 0
TerroristSpawn = 0

for element in stra:
  if (element[j] == "BombA"):
    BombA = BombA+1
  elif (element[j] == "BombB"):
    BombB = BombB+1
  elif (element[j] == "Mid"):
    Mid = Mid+1
  elif (element[j] == "Terroristspawn"):
    TerroristSpawn = TerroristSpawn + 1 
  if BombA >= 4:
    strategy = "RushA"
  if BombB >=4:
    strategy = "RushB"
  if BombA<4 & BombB<4:
    strategy = "Split"

Strategies.append(strategy)
print(Strategies)
    

  

print(BombA, BombB, Mid, TerroristSpawn, strategy)



In [ ]:
import matplotlib.pyplot as plt
from awpy.visualization.plot import plot_map

f, a = plot_map(map_name="de_dust2", map_type="simpleradar")
plt.show()

#vllt nochmal mit Liste versuchen, sorgt aber für Probleme hier, darum nicht mehr, aber code mal lassen

Mid = ["MidDoors", "TopofMid", "Middle", "CTSpawn", "Short"]
Short = ["ShortStairs", "UnderA", "ExtendedA", "Catwalk"]
BombB = ["BombsiteB", "BDoors", "Hole"]
TerroristSpawn = ["TSpawn", "TRamp"]
Tunnel = ["OutsideTunnel", "UpperTunnel", "TunnelStairs", "LowerTunnel"]
Long = ["LongA", "LongDoors", "OutsideLong", "Pit", "Side"]
BombA = ["BombsiteA", "ARamp","Ramp"]
i=0
po = []
for index, row in track4.iterrows():
  if track4.at[i,'area'] in Mid:
    e = "Mid"
    po.append(e)
    print(po)
    i = i+1
  elif track4.at[i,'area'] in Short:
    e = "Short"
    po.append(po)
    i = i+1
  elif track4.at[i,'area'] in BombB:
    e = "BombB"
    po.append(e)
    i = i+1
  elif track4.at[i,'area'] in TerroristSpawn:
    e = "TerroristSpawn"
    po.append(e)
    i = i+1
  elif track4.at[i,'area'] in Tunnel:
    e = "Tunnel"
    po.append(e)
    i = i+1
  elif track4.at[i,'area'] in Long:
    e = "Long"
    po.append(e)
    i = i+1
  elif track4.at[i,'area'] in BombA:
    e = "BombA"
    po.append(e)
    i = i+1
  else:
    e = "ERROR"
    po.append(e)
    i = i+1
  
track4["eeee"] = po